In [ ]:
import polars as pl
import numpy as np
import polars_ds

In [ ]:
test = pl.DataFrame({
    "x": ["a"] * 200 + ["b"] * 300 + ["c"] * 500
    , "y": [1] * 600 + [2] * 400
})
test.head()

In [ ]:
test.select(
    pl.col("x").stats.chi2(pl.col("y"))
)

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

In [ ]:
df2 = test.to_pandas()
contigency = pd.crosstab(index=df2['x'], columns=df2['y'])
chi2_contingency(
    contigency.to_numpy()
    , correction = True
)

In [ ]:
test.group_by(["x"]).agg(pl.count())

In [ ]:
t1 = test.select(
    pl.col("x").value_counts().struct.field("x"),
    pl.col("x").value_counts().struct.field("count")
)
t2 = test.select(
    pl.col("y").value_counts().struct.field("y"),
    pl.col("y").value_counts().struct.field("count")
)
t3 = test.group_by(["x", "y"]).agg(pl.count().alias("ob"))

fake_contigency = (
    t1.join(t2, left_on = "x", right_on = "y", how = "cross").select("x", "y")
    .join(t3, on = ["x", "y"], how = "left").with_columns(
        pl.col("ob").fill_null(0)
    )
)

fake_contigency.with_columns(
    (pl.col("ob").sum().over("y") * pl.col("ob").sum().over("x") / pl.col("ob").sum()).alias("e")
).select(
    ((pl.col("ob") - pl.col("e")).pow(2) / pl.col("e")).sum()
)


In [ ]:
t3

In [ ]:
test.select(
    pl.col("y").value_counts().struct.field("y"),
    pl.col("y").value_counts().struct.field("count")
)

In [ ]:
test.group_by(["x", "y"]).agg(
    pl.count().alias("observed")
).with_columns(
    pl.col("observed").sum().over("y").alias("y_sum"),
    pl.col("observed").sum().over("x").alias("x_sum")
).with_columns(
    ((pl.col("y_sum") * pl.col("x_sum"))/pl.col("observed").sum()).alias("expected")
)

In [ ]:
test.group_by(["x", "y"]).agg(
    pl.count().alias("observed")
).with_columns(
    (pl.col("observed").sum().over("y") * pl.col("observed").sum().over("x") / pl.col("observed").sum()).alias("expected")
).select(
    ((pl.col("observed") - pl.col("expected")).pow(2) / pl.col("expected")).sum()
)

In [ ]:
test.pivot(
    values = "x",
    index="x",
    columns="y",
    aggregate_function = "count"
)

In [ ]:
test.select(
    pl.col("a").str2.to_pascal_case()
)

In [ ]:
import numpy as np
from scipy import signal
rng = np.random.default_rng()

npoints = 10

noise = rng.standard_normal(npoints)

x = 3 + 2*np.linspace(0, 1, npoints) + noise

df = pl.DataFrame({
    "test": x,
    "entity_id": [1] *  5 + [2] * 5
})
df.head()

In [ ]:
import polars as pl
import polars_ds

df = pl.DataFrame({
    "test": x,
    "entity_id": [1] *  5 + [2] * 5
})
# And is 5x faster than Scipy.signal detrend on larger time series
df.select(
    pl.col("entity_id"),
    pl.col("test").num.detrend().over(pl.col("entity_id")).alias("test_detrended") # linear detrend
)

In [ ]:
%timeit signal.detrend(x)

In [ ]:
%timeit df.select(pl.col("test").num.detrend()).head()

In [ ]:
df.select(pl.col("test").num.detrend2()).head()

In [ ]:
df = pl.DataFrame({
    "c":[[0.1, 0.2], [0.5, 0.5], [-2, 2]],
    "z":[[0.1, 0.2], [0.5, 0.5], [-2, 2]]
})
df.head()

In [ ]:
df = pl.DataFrame({
    "a": list(range(10)) + [None] # Reference column
}).with_columns(
    pl.col("a").stats.rand_int(low=1., high=10, respect_null=True).alias("rand_int"),
    pl.col("a").stats.sample_uniform(low=1., high=3.).alias("uniform"),
    pl.col("a").stats.sample_normal(respect_null=True).alias("normal1"),
    pl.col("a").stats.sample_normal(mean = 2, std = 0.5).alias("normal2"),
    pl.col("a").stats.sample_exp(lam = 1.0).alias("exp"),
    pl.col("a").stats.sample_binomial(n = 10, p = 0.5).alias("binomial"),
    pl.col("a").stats.rand_str(min_size = 1, max_size = 10, respect_null=True).alias("rand_str")
)
df

In [ ]:
df.select(
    pl.col("a").stats.f_stats(pl.col("b"), pl.col("c"))
).item(0,0)

In [ ]:

df.select(
    pl.col("a").stats.f_test(pl.col("b"))
).item(0,0)

In [ ]:
from sklearn.feature_selection import f_regression, f_classif

In [ ]:
f_classif(df["b"].to_numpy().reshape(-1,1), df["a"].to_numpy())

In [ ]:
f_classif(df["c"].to_numpy().reshape(-1,1), df["a"].to_numpy())

In [ ]:
df.select(
    pl.col("a").str.sorensen_dice(pl.col("b"))
)

In [ ]:
df.select(
    pl.col("a").num.list_jaccard(pl.col("b"))
)

In [ ]:
df = pl.DataFrame({
    "a":["a", "b", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"],
    "b":["a", "b", "c", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d"]
})
df.head()

In [ ]:
df.select(
    pl.concat_str(pl.col("a"), pl.col("b"))
)

In [ ]:
df = pl.DataFrame({
    "a":["a", "b", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"],
    "b":["a", "b", "c", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d"]
})
vc = pl.col("a").value_counts(parallel=False, sort=True)
to_merge:pl.Expr = (
    vc.filter(
        vc.struct.field("counts") < 3
    ).struct.field("a")
)
df.select(to_merge)

In [ ]:
df.select(
    pl.col("b").str_ext.infer_infreq()
)